# Introduction

<p>In a United Parcel Service (UPS) package sorting facility, also known as a hub, packages that are not large enough to be sorted individually on the main conveyor belts are automatically sent to an area of the hub known as “small sort” to become “containerized.” This is the process of combining packages that have similar service levels and similar final delivery destinations into bags so that the bag can be routed to its delivery vehicle. The list of packages destined for the same bag will be referred to as a “load.”</p>
&nbsp;&nbsp;
<p>In most UPS hubs, a tilt-tray conveyor system is used to drop packages into the correct bags. When a bag becomes full, an employee needs to promptly zip up the bag, print out a label, place the label on the bag, place the bag onto a conveyor belt where it will be routed to the appropriate delivery vehicle, and place an empty bag in its place so that more packages can be dropped in that bin position. A “bin position” or “bin” is defined as a single position alongside the tilt tray where packages can be dropped and where bags are placed to catch them. The goal of this project is to present a mathematical model known as a linear program that effectively assigns loads to bin positions in a manner that maximizes the productivity of the employees processing the volume.</p>


Screenshot 2025-01-29 141616.png

# Part 1 - Mathematical Formulation of the Problem as a Linear Program
---

<p>An assignment linear programming (LP) problem is a type of optimization problem where the goal is to assign a set of tasks to a set of agents to minimize or maximize a linear objective function that is subject to linear constraints. In the load-to-bin assignment LP problem presented in this project, the objective function is the sum of the cost for every assignment made. Hence, the goal is to minimize this function while satisfying the constraints that every load be assigned to one bin position and every bin position be assigned one load.</p>

&nbsp;
## Assumptions and Initial Conditions

1. The information needed to model the problem is a list of loads that are awaiting a bin assignment and the corresponding expected volume (expected number of packages) for each of these loads. The expected volume data can be obtained from a UPS corporate database by a UPS industrial engineer. For data privacy, this data is generated randomly from a uniform distribution in this project.

2. The model assumes that the number of loads that need to be assigned is equal to the number of bins available. In the case where the number of loads awaiting assignment is less than the number of bins available, “dummy” loads are created that receive zero package volume to make the number of loads equal the number of bins. In the case where the number of loads awaiting assignment is greater than the number of bins available, the problem is declared to not be feasible, and the user will need to arbitrarily combine loads together to reduce the number of loads.

3. This formulation assumes that the number of employees working in the tilt-tray operations is known prior to solving the problem. This information is used during the process of computing appropriate cost coefficients for the linear objective function.


&nbsp;
## Components of the Model
&nbsp;
### Cost Coefficients
&nbsp;&nbsp;
<p>To create an objective function, a cost or penalty for every possible assignment is needed. For example, if load $l_i$ is assigned to bin $b_j$, we need to increase the objective function by a specfied amount. This value will be represented by $c_{ij}$.</p>
&nbsp;&nbsp;
<p>To compute the cost coefficients, we first introduce the concept of a "home position." These are the bin positions where we want employees to be stationed.</p>
&nbsp;&nbsp;
<p>The cost-coefficent $c_{ij}$ will be defined as the expected volume for load $l_i$ multiplied by the distance that bin position $b_j$ is from the nearest home position. Mathematically, we compute $c_{ij}$ as follows:</p>


* Create variables and sets to denote our important enviroment variables
  * Let $n$ represent the number of loads that need to be assigned to a bin position
  * Let $m$ represent the number of employees available to process packages on the tilt-tray
  * Let $L=\{l_1,l_2,…,l_n\}$ be the set of loads
  * Let $V=\{v_1,v_2,…,v_n\}$ be the set of volumes where $v_i$ is the expected volume for load $l_i$
  * The set $H$ will contain $m$ elements. The elements in set $H$ will denote which bin positions are the home positions. If $i$ is an element of $H$, then $b_i$ is a home position.
  * Let $D=\{d_1,d_2,…,d_n\}$ be the set of distances where $d_j$ represents the distance that $b_j$ is from the nearest home position.
* Select which bin position are to be home positions. We want these to be evenly spaced out across the set of bin positions.
  * Let $\phi=⌊\frac{n}{m}⌋$. This signifies that $\phi$ is the greatest integer less than or equal to the division of $𝑛$ by $𝑚$
  * Let $\psi =⌊\frac{n-(ϕ \cdot m)}{2}⌋$. This signifies that $\psi$ is the greatest integer less than or equal to the division of $(n-(ϕ∙m))$ by $2$
  * Let $H=\{h_1,h_2,…,h_m\}$ where $h_k=k \cdot \phi + ⌊\frac{ϕ}{2}⌋ + \psi$,$∀k∈\{1,2,…,m\}$
* Compute the distance that each bin position is from a home position
  * Let $d_j=\min⁡|h_k-j|,∀h_k∈H$
* Compute $c_{ij}$
  * Let $c_{ij}=v_i \cdot d_j$

&nbsp;
### Objective Function

The decision variables in the objective function will be represented by $x_{ij}$.

$$x_{ij} =
\begin{cases}
    1 & \text{if load } l_i \text{ is assigned to bin } b_j \\
    0 & \text{otherwise}
\end{cases}$$


The objective function containing cost coefficients and decision variables is as follows:

$$\min \sum_{i=1}^{n} \sum_{j=1}^{n} c_{ij} \cdot x_{ij}$$

&nbsp;
### Constraints

Every load must be assigned to only one bin:

$$\sum_{i=1}^{n} x_{ij}=1$$

Every bin must have only one load assigned to it:

$$\sum_{j=1}^{n} x_{ij}=1$$

Decision varibles must be non-negative:

$$x_{ij} \geq 0$$

Note: Because this is an assignment LP, we do not need to contrain our decision variables to be binary. This will result in the same result but at the expense of longer time for PuLP to solve the problem.


# Part 2 - Understanding the Model
---

### Why does this formulation produce an "optimal" load-to-bin assingment plan?

<p>
An "optimal" load-to-bin assignment plan is one in which employees can be most efficient processing bags in need of service. Being efficient in this context means spending more time productively replacing bags as they become full and less time traveling between bin positions to arrive at a bin in need of service.
</p>
&nbsp;
<p>
The model achieves such a plan by first choosing several bin positons to be "home positions." The number of home positions is equal to the total amount of employees set to be working in the tilt-tray operation. Home positions are spaced out at an equal distance from each other.
</p>
&nbsp;
<p>
The model then tries to place the loads with the highest expected volume closest to these home positions. The model achieves this because the cost coefficient in the linear objective function for load $l_i$ being assigned to bin $b_j$ is the expected volume for load $l_i$ (represented by $v_i$), multiplied by the distance that bin $b_j$ is from the nearest home position (represented by $d_j$). This effectively forces the optimization model to place loads with relatively high expected volume closer to home positions since the objective is to minimize the sum of the cost of all assignments.
</p>
&nbsp;
<p>
The idea is that each employee has a designated home position. In an optimal plan, the bin positions closest to home positions are the ones that will need to be serviced most frequently due to the loads assigned to these bin positions having relatively higher expected volume than loads assigned farther away from a home position. This layout requires less walking between bin positions to arrive at a bin in need of service and allows employees to spend the bulk of their time productively replacing bags as they become full.
</p>

# Part 3 - Python Implementation
---

We start with a problem with 100 loads needing an assignment and a plan for 10 employees to be working in the tilt-tray operation.

## Step 1: Generate a Dataframe for Loads

### Columns:

1. Load Name

  * Contains a list of load names.

2. Expected Volume

  * Contains the expected package volume for each load.

In [ ]:
import numpy as np
import pandas as pd
import random

# Parameters
num_loads = 100       # Number of loads needing bin assignment
low = 0               # Lowest Possible Expected Volume in uniform distribution
high = 200            # Highest Possible Expected Volume in uniform distribution

# Generate Column 1: Load Names
load_names = [f"Load {i+1}" for i in range(num_loads)]

# Generate Column 2: Expected Volume
expected_volume = [round(random.uniform(low, high),2) for _ in range(num_loads)]

# Create the DataFrame
loads_df = pd.DataFrame({
    "Load Name": load_names,
    "Expected Volume": expected_volume,
})

loads_df.head(20)

Load Name  Expected Volume
0     Load 1            29.20
1     Load 2            10.97
2     Load 3            47.46
3     Load 4            91.13
4     Load 5           141.30
5     Load 6            52.99
6     Load 7            45.00
7     Load 8            10.90
8     Load 9             8.91
9    Load 10            96.75
10   Load 11            94.62
11   Load 12            23.95
12   Load 13            89.58
13   Load 14           127.37
14   Load 15           134.18
15   Load 16           163.69
16   Load 17           194.80
17   Load 18            95.56
18   Load 19           156.28
19   Load 20             5.13

## Step 2: Generate a Dataframe for Bin Positions

### Columns:

1. Bin Position Name

  * This column contains a list of bin position names.

2. Distance

  * The value in this column represents the distance (in terms of number of bin positions) that the corresponding bin position is from a home position.
  * These values are important in the next step of model, creating a cost coefficeint matrix.


In [ ]:
# Parameters
num_bins = num_loads  # Number of bins is assumed to be equal to the number of loads needing bin assignment
num_employees = 5     # Number of employees working on tilt-tray operation

# Generate Column 1: Bin Position Names
bin_names = [f"Bin {i+1}" for i in range(num_bins)]

# Generate Column 2: Distances
home_position = np.zeros(num_employees, dtype=int)
interval = num_bins // num_employees
start_point = (num_bins - (interval * num_employees)) // 2

for k in range(num_employees):
    home_position[k] = start_point + k * interval + interval // 2

print("The following bin positions are the home positions: ",home_position)

distances = np.zeros(num_bins)

for j in range(0, num_bins):
  min_distance = np.inf
  for h_k in home_position:
    min_distance = min(min_distance, abs(h_k - j))
  distances[j] = min_distance

# Create the DataFrame
bins_df = pd.DataFrame({
    "Bin Name": bin_names,
    "Distance": distances
})

bins_df.head(20)

The following bin positions are the home positions:  [10 30 50 70 90]


Bin Name  Distance
0     Bin 1      10.0
1     Bin 2       9.0
2     Bin 3       8.0
3     Bin 4       7.0
4     Bin 5       6.0
5     Bin 6       5.0
6     Bin 7       4.0
7     Bin 8       3.0
8     Bin 9       2.0
9    Bin 10       1.0
10   Bin 11       0.0
11   Bin 12       1.0
12   Bin 13       2.0
13   Bin 14       3.0
14   Bin 15       4.0
15   Bin 16       5.0
16   Bin 17       6.0
17   Bin 18       7.0
18   Bin 19       8.0
19   Bin 20       9.0

## Step 3: Creating a Cost Coefficent Matrix


In [ ]:
# Create Cost Coefficent Matrix

load_vol = loads_df['Expected Volume']
multiplier = bins_df['Distance']

matrix = np.zeros((num_loads, num_bins))

for i in range(num_loads):
    for j in range(num_bins):
      matrix[i, j] = multiplier[i]*load_vol[j]

cost_coefficients_df = pd.DataFrame(matrix, columns=[f'Load {i+1}' for i in range(num_loads)], index=[f'Bin {i+1}' for i in range(num_bins)])

cost_coefficients_df.head(20)

Load 1  Load 2  Load 3  Load 4  Load 5  Load 6  Load 7  Load 8  Load 9  Load 10  Load 11  \
Bin 1    292.0  109.70  474.60  911.30  1413.0  529.90   450.0   109.0   89.10   967.50   946.20   
Bin 2    262.8   98.73  427.14  820.17  1271.7  476.91   405.0    98.1   80.19   870.75   851.58   
Bin 3    233.6   87.76  379.68  729.04  1130.4  423.92   360.0    87.2   71.28   774.00   756.96   
Bin 4    204.4   76.79  332.22  637.91   989.1  370.93   315.0    76.3   62.37   677.25   662.34   
Bin 5    175.2   65.82  284.76  546.78   847.8  317.94   270.0    65.4   53.46   580.50   567.72   
Bin 6    146.0   54.85  237.30  455.65   706.5  264.95   225.0    54.5   44.55   483.75   473.10   
Bin 7    116.8   43.88  189.84  364.52   565.2  211.96   180.0    43.6   35.64   387.00   378.48   
Bin 8     87.6   32.91  142.38  273.39   423.9  158.97   135.0    32.7   26.73   290.25   283.86   
Bin 9     58.4   21.94   94.92  182.26   282.6  105.98    90.0    21.8   17.82   193.50   189.24   
Bin 10    29.2   10.97   47.46   91.13   141.3   52.99    45.0    10.9    8.91    96.75    94.62   
Bin 11     0.0    0.00    0.00    0.00     0.0    0.00     0.0     0.0    0.00     0.00     0.00   
Bin 12    29.2   10.97   47.46   91.13   141.3   52.99    45.0    10.9    8.91    96.75    94.62   
Bin 13    58.4   21.94   94.92  182.26   282.6  105.98    90.0    21.8   17.82   193.50   189.24   
Bin 14    87.6   32.91  142.38  273.39   423.9  158.97   135.0    32.7   26.73   290.25   283.86   
Bin 15   116.8   43.88  189.84  364.52   565.2  211.96   180.0    43.6   35.64   387.00   378.48   
Bin 16   146.0   54.85  237.30  455.65   706.5  264.95   225.0    54.5   44.55   483.75   473.10   
Bin 17   175.2   65.82  284.76  546.78   847.8  317.94   270.0    65.4   53.46   580.50   567.72   
Bin 18   204.4   76.79  332.22  637.91   989.1  370.93   315.0    76.3   62.37   677.25   662.34   
Bin 19   233.6   87.76  379.68  729.04  1130.4  423.92   360.0    87.2   71.28   774.00   756.96   
Bin 20   262.8   98.73  427.14  820.17  1271.7  476.91   405.0    98.1   80.19   870.75   851.58   

        Load 12  Load 13  Load 14  Load 15  Load 16  Load 17  Load 18  Load 19  Load 20  Load 21  \
Bin 1    239.50   895.80  1273.70  1341.80  1636.90   1948.0   955.60  1562.80    51.30   241.90   
Bin 2    215.55   806.22  1146.33  1207.62  1473.21   1753.2   860.04  1406.52    46.17   217.71   
Bin 3    191.60   716.64  1018.96  1073.44  1309.52   1558.4   764.48  1250.24    41.04   193.52   
Bin 4    167.65   627.06   891.59   939.26  1145.83   1363.6   668.92  1093.96    35.91   169.33   
Bin 5    143.70   537.48   764.22   805.08   982.14   1168.8   573.36   937.68    30.78   145.14   
Bin 6    119.75   447.90   636.85   670.90   818.45    974.0   477.80   781.40    25.65   120.95   
Bin 7     95.80   358.32   509.48   536.72   654.76    779.2   382.24   625.12    20.52    96.76   
Bin 8     71.85   268.74   382.11   402.54   491.07    584.4   286.68   468.84    15.39    72.57   
Bin 9     47.90   179.16   254.74   268.36   327.38    389.6   191.12   312.56    10.26    48.38   
Bin 10    23.95    89.58   127.37   134.18   163.69    194.8    95.56   156.28     5.13    24.19   
Bin 11     0.00     0.00     0.00     0.00     0.00      0.0     0.00     0.00     0.00     0.00   
Bin 12    23.95    89.58   127.37   134.18   163.69    194.8    95.56   156.28     5.13    24.19   
Bin 13    47.90   179.16   254.74   268.36   327.38    389.6   191.12   312.56    10.26    48.38   
Bin 14    71.85   268.74   382.11   402.54   491.07    584.4   286.68   468.84    15.39    72.57   
Bin 15    95.80   358.32   509.48   536.72   654.76    779.2   382.24   625.12    20.52    96.76   
Bin 16   119.75   447.90   636.85   670.90   818.45    974.0   477.80   781.40    25.65   120.95   
Bin 17   143.70   537.48   764.22   805.08   982.14   1168.8   573.36   937.68    30.78   145.14   
Bin 18   167.65   627.06   891.59   939.26  1145.83   1363.6   668.92  1093.96    35.91   169.33   
Bin 19 

## Step 4: Solving the Linear Program

In [ ]:
!pip install pulp
from pulp import *

In [ ]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum

# Define the problem
prob = LpProblem("AssignmentProblem", LpMinimize)

# Define the decision variables
x = LpVariable.dicts("x", [(i, j) for i in range(num_bins) for j in range(num_loads)], lowBound=0, cat='Continuous')

# Objective function: Minimize the total cost
prob += lpSum([cost_coefficients_df.iloc[i, j] * x[(i, j)] for i in range(num_bins) for j in range(num_loads)])

# Constraints: Each bin is assigned to exactly one load
for i in range(num_bins):
    prob += lpSum([x[(i, j)] for j in range(num_loads)]) == 1

# Constraints: Each load is assigned to exactly one bin
for j in range(num_loads):
    prob += lpSum([x[(i, j)] for i in range(num_bins)]) == 1

# Solve the problem
prob.solve()

# Extract the assignments
assignments = [(i, j) for i in range(num_bins) for j in range(num_loads) if x[(i, j)].varValue == 1]

# Create a DataFrame to display the assignments
result_df = pd.DataFrame(assignments, columns=['Bin', 'Load'])

# Add Bins column with numbers counting from 1 to num_bins
result_df['Bin Position'] = result_df['Bin'] + 1

# Add Load names and Expected Volume from the loads_df
result_df['Load Name'] = result_df['Load'].apply(lambda x: loads_df.at[x, 'Load Name'])
result_df['Expected Volume'] = result_df['Load'].apply(lambda x: loads_df.at[x, 'Expected Volume'])

# Rearrange columns
result_df = result_df[['Bin Position', 'Load Name', 'Expected Volume']]

# Create horizontal bars starting from the left
max_length = 20  # Maximum length of the bar
max_value = result_df['Expected Volume'].max()
result_df['Bar'] = result_df['Expected Volume'].apply(lambda x: '█' * int(max_length * (x / max_value)))

# Rearrange columns
result_df = result_df[['Bin Position', 'Load Name', 'Expected Volume', 'Bar']]

# Set options to display the entire DataFrame
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Adjust the width to the screen's width
pd.set_option('display.max_colwidth', None)  # Show full column content

# Display the DataFrame
result_df



Bin Position Load Name  Expected Volume                   Bar
0              1   Load 55             5.94                      
1              2    Load 8            10.90                     █
2              3   Load 54            27.01                    ██
3              4   Load 30            41.31                  ████
4              5   Load 87            65.37                ██████
5              6   Load 65            85.79              ████████
6              7    Load 4            91.13             █████████
7              8   Load 56           136.28         █████████████
8              9   Load 35           151.09       ███████████████
9             10   Load 84           190.80   ███████████████████
10            11  Load 100           195.34   ███████████████████
11            12   Load 94           177.48     █████████████████
12            13   Load 46           152.61       ███████████████
13            14   Load 15           134.18         █████████████
14            15   Load 79            98.19             █████████
15            16   Load 41            76.73               ███████
16            17   Load 23            57.96                 █████
17            18   Load 72            34.93                   ███
18            19   Load 97            19.24                     █
19            20   Load 78             9.88                      
20            21   Load 60             4.12                      
21            22   Load 96            17.12                     █
22            23   Load 12            23.95                    ██
23            24   Load 70            36.91                   ███
24            25   Load 37            65.62                ██████
25            26   Load 71            89.12              ████████
26            27   Load 25            93.09             █████████
27            28    Load 5           141.30        ██████████████
28            29   Load 16           163.69      ████████████████
29            30   Load 59           173.58     █████████████████
30            31   Load 73           195.06   ███████████████████
31            32   Load 82           190.22   ███████████████████
32            33   Load 47           161.26      ████████████████
33            34   Load 14           127.37          ████████████
34            35   Load 90           112.78           ███████████
35            36   Load 42            68.78                ██████
36            37   Load 50            55.93                 █████
37            38   Load 26            40.49                  ████
38            39   Load 21            24.19                    ██
39            40    Load 2            10.97                     █
40            41   Load 81             2.19                      
41            42   Load 53             6.21                      
42            43   Load 69            27.01                    ██
43            44    Load 3            47.46                  ████
44            45   Load 31            65.08                ██████
45            46   Load 38            87.74              ████████
46            47   Load 93            90.13             █████████
47            48   Load 24           112.83           ███████████
48            49   Load 58           152.43       ███████████████
49            50   Load 89           191.53   ███████████████████
50            51   Load 66           195.05   ███████████████████
51            52   Load 63           194.04   ███████████████████
52            53   Load 19           156.28       ███████████████
53            54   Load 74           131.04         █████████████
54            55   Load 88            93.77             █████████
55            56   Load 95            89.40              ████████
56            57   Load 99            65.70                ██████
57            58   Load 34            40.94                  ████
58            59    Load 1            29.20                    ██
59            60   Load 29             9.55 

# Interpretation
---

In the visual above, we can see that the model has successfully created an assignment plan that places the highest volume loads nearest to the home positions. It is evident from the visual above that the bars (which depict the relative load volumes) progressively get larger until a home position is reached. After the peak, these bars progressively decrease. Once the next homepostion is approaching, the bars get progressively larger to build up to the next peak. This is the result we want.

# Scaling to a Larger Problem
* 1000 loads
* 20 employees




In [ ]:
import numpy as np
import pandas as pd
import random

# Parameters
num_loads = 1000       # Number of loads needing bin assignment
low = 0               # Lowest Possible Expected Volume in uniform distribution
high = 200            # Highest Possible Expected Volume in uniform distribution

# Generate Column 1: Load Names
load_names = [f"Load {i+1}" for i in range(num_loads)]

# Generate Column 2: Expected Volume
expected_volume = [round(random.uniform(low, high),2) for _ in range(num_loads)]

# Create the DataFrame
loads_df = pd.DataFrame({
    "Load Name": load_names,
    "Expected Volume": expected_volume,
})

In [ ]:
# Parameters
num_bins = num_loads  # Number of bins is assumed to be equal to the number of loads needing bin assignment
num_employees = 20     # Number of employees working on tilt-tray operation

# Generate Column 1: Bin Position Names
bin_names = [f"Bin {i+1}" for i in range(num_bins)]

# Generate Column 2: Distances
home_position = np.zeros(num_employees, dtype=int)
interval = num_bins // num_employees
start_point = (num_bins - (interval * num_employees)) // 2

for k in range(num_employees):
    home_position[k] = start_point + k * interval + interval // 2

print("The following bin positions are the home positions: ",home_position)

distances = np.zeros(num_bins)

for j in range(0, num_bins):
  min_distance = np.inf
  for h_k in home_position:
    min_distance = min(min_distance, abs(h_k - j))
  distances[j] = min_distance

# Create the DataFrame
bins_df = pd.DataFrame({
    "Bin Name": bin_names,
    "Distance": distances
})

The following bin positions are the home positions:  [ 25  75 125 175 225 275 325 375 425 475 525 575 625 675 725 775 825 875
 925 975]


In [ ]:
# Create Cost Coefficent Matrix

load_vol = loads_df['Expected Volume']
multiplier = bins_df['Distance']

matrix = np.zeros((num_loads, num_bins))

for i in range(num_loads):
    for j in range(num_bins):
      matrix[i, j] = multiplier[i]*load_vol[j]

cost_coefficients_df = pd.DataFrame(matrix, columns=[f'Load {i+1}' for i in range(num_loads)], index=[f'Bin {i+1}' for i in range(num_bins)])

In [ ]:
from pulp import LpMinimize, LpProblem, LpVariable, lpSum

# Define the problem
prob = LpProblem("AssignmentProblem", LpMinimize)

# Define the decision variables
x = LpVariable.dicts("x", [(i, j) for i in range(num_bins) for j in range(num_loads)], lowBound=0, cat='Continuous')

# Objective function: Minimize the total cost
prob += lpSum([cost_coefficients_df.iloc[i, j] * x[(i, j)] for i in range(num_bins) for j in range(num_loads)])

# Constraints: Each bin is assigned to exactly one load
for i in range(num_bins):
    prob += lpSum([x[(i, j)] for j in range(num_loads)]) == 1

# Constraints: Each load is assigned to exactly one bin
for j in range(num_loads):
    prob += lpSum([x[(i, j)] for i in range(num_bins)]) == 1

# Solve the problem
prob.solve()

# Extract the assignments
assignments = [(i, j) for i in range(num_bins) for j in range(num_loads) if x[(i, j)].varValue == 1]

# Create a DataFrame to display the assignments
result_df = pd.DataFrame(assignments, columns=['Bin', 'Load'])

# Add Bins column with numbers counting from 1 to num_bins
result_df['Bin Position'] = result_df['Bin'] + 1

# Add Load names and Expected Volume from the loads_df
result_df['Load Name'] = result_df['Load'].apply(lambda x: loads_df.at[x, 'Load Name'])
result_df['Expected Volume'] = result_df['Load'].apply(lambda x: loads_df.at[x, 'Expected Volume'])

# Rearrange columns
result_df = result_df[['Bin Position', 'Load Name', 'Expected Volume']]

# Create horizontal bars starting from the left
max_length = 20  # Maximum length of the bar
max_value = result_df['Expected Volume'].max()
result_df['Bar'] = result_df['Expected Volume'].apply(lambda x: '█' * int(max_length * (x / max_value)))

# Rearrange columns
result_df = result_df[['Bin Position', 'Load Name', 'Expected Volume', 'Bar']]

# Set options to display the entire DataFrame
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Adjust the width to the screen's width
pd.set_option('display.max_colwidth', None)  # Show full column content

# Display the DataFrame
result_df

Bin Position  Load Name  Expected Volume                   Bar
0               1   Load 712             3.25                      
1               2   Load 645             8.12                      
2               3    Load 21            15.44                     █
3               4   Load 735            26.68                    ██
4               5   Load 132            29.44                    ██
5               6   Load 898            38.54                   ███
6               7   Load 604            47.03                  ████
7               8   Load 692            57.45                 █████
8               9   Load 710            61.92                ██████
9              10   Load 406            77.07               ███████
10             11   Load 225            83.61              ████████
11             12   Load 905            91.26             █████████
12             13   Load 171           100.38            ██████████
13             14    Load 61           104.22            ██████████
14             15   Load 284           115.29           ███████████
15             16   Load 105           117.95           ███████████
16             17   Load 842           126.74          ████████████
17             18   Load 208           136.49         █████████████
18             19    Load 24           146.50        ██████████████
19             20   Load 377           151.03       ███████████████
20             21   Load 340           160.47      ████████████████
21             22   Load 953           169.28      ████████████████
22             23   Load 166           181.50    ██████████████████
23             24   Load 616           189.48    ██████████████████
24             25   Load 481           196.60   ███████████████████
25             26   Load 840           197.86   ███████████████████
26             27   Load 281           193.44   ███████████████████
27             28   Load 464           182.71    ██████████████████
28             29   Load 650           181.81    ██████████████████
29             30   Load 656           170.15     █████████████████
30             31   Load 500           159.80      ████████████████
31             32    Load 41           148.88        ██████████████
32             33   Load 314           147.99        ██████████████
33             34     Load 1           132.59         █████████████
34             35   Load 703           130.23         █████████████
35             36   Load 974           123.58          ████████████
36             37   Load 981           111.27           ███████████
37             38   Load 825           104.73            ██████████
38             39   Load 477            94.99             █████████
39             40   Load 198            91.88             █████████
40             41   Load 793            82.91              ████████
41             42   Load 215            69.85               ███████
42             43   Load 438            64.16                ██████
43             44    Load 30            55.51                 █████
44             45   Load 796            46.85                  ████
45             46   Load 410            35.46                   ███
46             47   Load 716            33.75                   ███
47             48   Load 575            24.45                    ██
48             49   Load 248            18.17                     █
49             50   Load 752            11.05                     █
50             51   Load 580             0.73                      
51             52   Load 769             7.93                      
52             53   Load 852            18.56                     █
53             54   Load 236            21.31                    ██
54             55   Load 718            31.02                   ███
55             56   Load 505            42.59                  ████
56             57   Load 906            50.06                 █████
57             58   Load 896            52.55                